In [1]:
import pandas as pd
import numpy as np
import io
import sys
import os.path
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
df = pd.read_csv("Data/data_with_path.csv",encoding='ISO-8859-1')
df.head()

,Title,Release_Year,Genre,Synopsis,Poster_URL,Movie_ID,Movie_URL,Duration,Certification,Voters,Rating,Path
0,The Medium,(2021),"Drama,War",An Ex-French Soldier begins participating in underground street fights in order to make money for his brother's family.,"https://m.media-amazon.com/images/M/MV5BNDA4ZDVlOWMtNzAzMi00YmYwLTljYjctYzQ1MzcyOWZmYzdmXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_UX67_CR0,0,67,98_AL_.jpg",100029,https://www.imdb.com/title/tt0100029/,108 min,A,"37,253",6.2,Posters\100029.jpg
1,The Unforgivable,(2021),Comedy,Two couples rent a vacation home for what should be a celebratory weekend get-away.,"https://m.media-amazon.com/images/M/MV5BMjMwOGEyMTQtMDllMC00YTU3LTgyNzktYmEwY2Y4MjQzYmRkXkEyXkFqcGdeQXVyMjM4OTYzMTQ@._V1_UX67_CR0,0,67,98_AL_.jpg",10003008,https://www.imdb.com/title/tt10003008/,52 min,R,"18,464",5.9,Posters\10003008.jpg
2,Hidden,(I) (2015),"Horror,Sci-Fi,Thriller",Based on the popular diary of the anonymous sex worker known only as Belle de Jour.,"https://m.media-amazon.com/images/M/MV5BMTM1Njg3MzgwOF5BMl5BanBnXkFtZTcwNzMxMzU3MQ@@._V1_UY98_CR5,0,67,98_AL_.jpg",1000734,https://www.imdb.com/title/tt1000734/,43 min,UA,"158,121",7.5,Posters\1000734.jpg
3,Royal Pains,(2022â ),"Action,Crime,Drama",A New York City writer on sex and love is finally getting married to her Mr. Big. But her three best girlfriends must console her after one of them inadvertently leads Mr. Big to jilt her.,"https://m.media-amazon.com/images/M/MV5BMTYyMzYxMjM3OF5BMl5BanBnXkFtZTcwNjg2OTA3MQ@@._V1_UX67_CR0,0,67,98_AL_.jpg",1000774,https://www.imdb.com/title/tt1000774/,96 min,15+,"92,249",5.7,Posters\1000774.jpg
4,Blood of Zeus,(2020â ),"Animation,Action,Adventure","A commoner living in ancient Greece, Heron discovers his true heritage as a son of Zeus, and his purpose: to save the world from a demonic army.","https://m.media-amazon.com/images/M/MV5BOWEzZmI2NTItMTFhZi00OWY0LWJiYTktMzIzYzRhNjA1N2IyXkEyXkFqcGdeQXVyMTUzMTM1NDU@._V1_UY98_CR15,0,67,98_AL_.jpg",10009170,https://www.imdb.com/title/tt10009170/,30 min,18,"18,832",7.5,Posters\10009170.jpg


In [3]:
df.shape

(6914, 12)

In [4]:
# There exists a genre value for all images
df['Genre'] = df['Genre'].str.replace(" ", "")
Genre_list = df['Genre']
print(len(Genre_list))

6914


In [5]:
# Breaks "Genre" into the constituting individual genres
def find_genres(genre):
    
    start = 0
    set_of_genre = []
    for i in range(len(genre)):
        
        k=0
        substring = ""
        if (genre[i]==','):
            substring = genre[start:i]
            start = i+1
            k = 1
        
        if(i==len(genre)-1):
            substring = genre[start:i+1]
            k = 1
            
        if (k==1):
            set_of_genre.append(substring)         
    
    return (set_of_genre)

In [6]:
# Extract list of genre values for each image
all_genre = []
Genre_list = df['Genre']

for i in range (len(Genre_list)):
    
    set_of_genre = find_genres(Genre_list[i])
    
    for j in range (len(set_of_genre)):
        all_genre.append(set_of_genre[j])
        
uniq, counts = np.unique(all_genre, return_counts=True)
print("Number of unique genres:", len(uniq))
print("Unique genres are:", uniq)
dict(zip(uniq, counts))

Number of unique genres: 28
Unique genres are: ['A' 'Action' 'Adult' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Game-Show' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'News' 'Reality-TV' 'Romance' 'Sci-Fi'
 'Short' 'Sport' 'Talk-Show' 'Thriller' 'War' 'Western']


{'A': 3,
 'Action': 1617,
 'Adult': 2,
 'Adventure': 1227,
 'Animation': 468,
 'Biography': 477,
 'Comedy': 2386,
 'Crime': 1388,
 'Documentary': 174,
 'Drama': 3942,
 'Family': 362,
 'Fantasy': 521,
 'Game-Show': 39,
 'History': 236,
 'Horror': 674,
 'Music': 181,
 'Musical': 58,
 'Mystery': 814,
 'News': 4,
 'Reality-TV': 98,
 'Romance': 967,
 'Sci-Fi': 457,
 'Short': 71,
 'Sport': 142,
 'Talk-Show': 10,
 'Thriller': 974,
 'War': 69,
 'Western': 50}

In [7]:
df = pd.read_csv("Data/data_with_path.csv",encoding='ISO-8859-1')

In [8]:
# Prepare multi-hot-encoded-labels for the various genres
def multi_hot_encoded_labels(img_id, genre):
    
    col_names =  ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']
    
    set_of_genre = find_genres(genre)
    
    row=[]
    row.append(img_id)
    
    for i in range(len(col_names)):
        
        found = 0
        for j in range (len(set_of_genre)):
            if (set_of_genre[j]==col_names[i]):
                found = 1
                break
        
        row.append(found)
    
    row.append(genre) #add the overall combined genre for record purposes
        
    return row  

In [9]:
# Perform the encoding of the labels and save data in the format :
# Img-ID <multi-hot-encoded-labels> overall_genre
all_data = []

for index, row in tqdm(df.iterrows()):
    
    path = row['Path']
    genre = row['Genre']
    row = multi_hot_encoded_labels(path, genre)
    
    all_data.append(row)

col_names =  ['Img-paths', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'Genre']

np.savetxt("Data/Multi_hot_encoded_data.csv", np.asarray(all_data), fmt='%s', delimiter=" ")
np.savetxt("Data/Encoded_data_column_lookup.csv", np.asarray(col_names), fmt='%s', delimiter=" ")

6914it [00:00, 12627.10it/s]


In [10]:
df_encoded = pd.read_csv("Data/Multi_hot_encoded_data.csv", delimiter=" ", 
                  names =  ['Img-paths', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'Genre'])

df_encoded.head()

,Img-paths,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,Genre
0,Posters\100029.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,"Drama,War"
1,Posters\10003008.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
2,Posters\1000734.jpg,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,"Horror,Sci-Fi,Thriller"
3,Posters\1000774.jpg,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,"Action,Crime,Drama"
4,Posters\10009170.jpg,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Animation,Action,Adventure"


In [11]:
# To split into train / validation / test in the ratio 80 / 15 / 5%

df = pd.read_csv("Data/Multi_hot_encoded_data.csv", delimiter=" ")
random_seed = 50
train_df = df.sample(frac=0.70, random_state=random_seed) #Taking 70% of the data
tmp_df = df.drop(train_df.index)
test_df = tmp_df.sample(frac=0.1, random_state=random_seed) #Taking 20% of the remaining (after train is taken)
valid_df = tmp_df.drop(test_df.index)

print("Train_df=",len(train_df))
print("Val_df=",len(valid_df))
print("Test_df=",len(test_df))

np.savetxt("Data/Train.csv", train_df, fmt='%s', delimiter=" ")
np.savetxt("Data/Test.csv", test_df, fmt='%s', delimiter=" ")
np.savetxt("Data/Valid.csv", valid_df, fmt='%s', delimiter=" ")

Train_df= 4839
Val_df= 1867
Test_df= 207
